In [1]:
import cv2
import numpy as np
import keying
import projection
import imp
import time

In [32]:
import line_profiler

In [2]:
imp.reload(keying)

<module 'keying' from '/home/ozym4nd145/Coding/Project/Vision_Project/keying.py'>

In [3]:
def show_images(lst):
    while True:
        for i in lst:
            cv2.imshow(i[0],i[1])
        ret = cv2.waitKey(0)
        if ret==27:
            break
    cv2.destroyAllWindows()

In [43]:
def project_img(img, bg, mask_corners, key_param, tola=16, tolb=50, low_thresh=0.05, high_thresh=0.25,
                erode_sz=3, sz=5, space=200, sat_mul_lo=5, sat_mul_hi=7, scale_blur=20, blur_size=3):
    
    key_mask = keying.get_mask(
        img, key_param[0], tola, tolb, low_thresh, high_thresh, sz, space, erode_sz)
    
    if not (sat_mul_hi == 0):
        mod_img = keying.process_img(img, key_param[1], sat_mul_lo, sat_mul_hi)
    else:
        mod_img = img

    bgra = keying.get_bgra(mod_img, key_mask)

    res = projection.project_to_mask(bgra, mask_corners, bg.shape)
    
    trans_img = res[:, :, :3]  # (x,y,3)
    trans_mask = np.expand_dims(
        (res[:, :, 3]).astype(np.float32) / 255, -1)  # (x,y)
    
    result = trans_img*trans_mask + bg*(1-trans_mask)
    
    if not (scale_blur <= 0 or blur_size<=1):
        light_mask = np.expand_dims(
            scale_blur * cv2.blur(trans_mask * (1 - trans_mask), (blur_size, blur_size)), -1)
        light_mask = np.clip(light_mask, 0.0, 1.0)
        light = bg * light_mask
        diffl = result * (1 - light_mask)
        light_result = (light + diffl).astype(np.uint8)
    else:
        light_result = result

    end_time = time.time()
    return light_result

In [5]:
def get_key_param(img):
    key_region = keying.get_region(img)
    ycrcb = keying.get_params_ycrcb(img,key_region)
    hls = keying.get_params_hls(img,key_region)
    return (ycrcb,hls)

## IMG Test

In [ ]:
img = cv2.imread("./image/twoperson.png")
bg = cv2.imread("./image/bg.jpg")
mask = cv2.imread("./image/mask.jpg")

key_param = get_key_param(img)
mask_corner = projection.get_corners_mask(mask)
res,m = project_img(img,bg,mask_corner,key_param)

show_images([("i",img),("bg",bg),("m",mask),("res",res)])

## Video Test

In [6]:
green_vid_path = "./video/fight_green.mp4"
# green_vid_path = "./video/suyash_oneplus.mp4"
env_vid_path = "./video/fight.avi"
env_mask_path = "./video/fight_mask.txt"
outp_vid = "./video/fight_result.mov"
outp_vid_l = "./video/fight_result_light.mov"

In [7]:
cap_green = cv2.VideoCapture(green_vid_path)
cap_vid = cv2.VideoCapture(env_vid_path)

In [8]:
ret,img = cap_green.read()
# img = np.transpose(img,[1,0,2])
ret,bg = cap_vid.read()
with open(env_mask_path,"r") as fl:
    lines = fl.read().split("\n")[:-1]
    corners = []
    for l in lines:
        p = [int(a) for a in l.split()]
        corners.append([[p[0],p[1]],[p[2],p[3]],[p[4],p[5]],[p[6],p[7]]])
    corners = np.array(corners,dtype=np.float32)

In [9]:
key_param = get_key_param(img)

In [10]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = cap_vid.get(5)
vid = cv2.VideoWriter(outp_vid,fourcc, fps, (bg.shape[1],bg.shape[0]))

In [11]:
iteration = 0
end_iter = len(corners)

In [14]:
while True:
    start_t = time.time()
    ret1,img = cap_green.read()
    ret2,bg = cap_vid.read()
    
    if (not(ret1 and ret2 and(iteration != end_iter))):
        break
        
#     res = project_img(img,bg,corners[iteration],key_param)
    res = project_img(img, bg, corners[iteration], key_param, tola = 16 ,tolb = 50 ,
                          low_thresh = 0.05 ,high_thresh = 0.25 ,erode_sz = 0 ,
                          sz = 0 ,space = 0 ,sat_mul_lo = 0 ,sat_mul_hi = 0 ,
                          scale_blur = 0 ,blur_size = 0)
#     vid.write(res)
    
    end_time = time.time()
    print(end_time-start_t)
    
    cv2.imshow("result",res)
    ret = cv2.waitKey(20)
    if ret==27:
        break
    
    iteration += 1
cv2.destroyAllWindows()
vid.release()

0.20133662223815918
0.1522974967956543
0.13595247268676758
0.1395552158355713
0.14001131057739258
0.13579726219177246
0.13793587684631348
0.13484454154968262
0.13934755325317383
0.13616251945495605
0.16551947593688965
0.14013409614562988
0.13527297973632812
0.13132333755493164
0.13773465156555176
0.1269087791442871
0.13303327560424805
0.1325390338897705
0.128753662109375
0.12716341018676758
0.14754724502563477
0.12554168701171875
0.13181161880493164
0.13381528854370117
0.13975977897644043


NameError: name 'livid' is not defined

In [40]:
%timeit -n 10 -r 2 project_img(img, bg, corners[iteration], key_param)

10 loops, best of 2: 185 ms per loop


In [41]:
%lprun -f project_img project_img(img, bg, corners[iteration], key_param, tola = 16 ,tolb = 50 ,low_thresh = 0.05 ,high_thresh = 0.25 ,erode_sz = 0 ,sz = 0 ,space = 0 ,sat_mul_lo = 0 ,sat_mul_hi = 0 ,scale_blur = 0 ,blur_size = 0)

In [42]:
%lprun -f project_img project_img(img, bg, corners[iteration], key_param)

In [67]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
